In [1]:
data_file = "training.1600000.processed.noemoticon.csv"
from csv import reader
# open file in read mode
count = 0
pos = []
#c2 = []
neg = []

words_to_use = ["love", "hate", "aww", "bad", "good", "great", "fuck", "jesus", "sucks", "suck", "pleased", 
                "fanntastic", "amazing", "enjoy", "excellent", "day", "today", "yesterday", "dad", "mom",
                "kid", "child", "year", "think", "way", "first", "well", "even", "new", "any", "most", "man", "boy",
                "woman", "girl", "time", "person", "sad", "happy", "yes", "no", "well", "yea", "yeah", "hell", "sure",
               "ok", "wife", "husband", "kill", "ill", "sick", "illness", "sickness", "death", "virus", "dead", "killed", 
               "accident"]

with open(data_file, 'r', encoding='mac_roman') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        st = ([s.strip().lower() for s in row])
        cur_str = []
        found = False
        for w in st[-1].split(' '):
            if len(w) > 0:
                if w in words_to_use:
                    found = True
                    if w[0] not in ['@']:
                        cur_str.append(w.replace("!", "").replace("?", "").replace(".", ""))

        if found:
            if (st[0]) == '0':
                neg.append(" ".join(cur_str))
#            if (st[0]) == '2':
#                c2.append(" ".join(cur_str))
            if (st[0]) == '4':
                pos.append(" ".join(cur_str))

In [25]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words={'english'})
X = vectorizer.fit_transform(neg + pos).toarray()
train_words = (vectorizer.get_feature_names())
p_y1 = len(neg) / (len(pos) + len(neg))
p_y2 = len(pos) / (len(pos) + len(neg))
print(X.shape)
print(len(train_words))

(604386, 57)
57


In [ ]:
x1 = vectorizer.transform(neg)
m1 = x1.mean(axis=0).flatten()
x2 = vectorizer.transform(pos)
m2 = x2.mean(axis=0).flatten()

print("Neutral words")
for iw, w in enumerate(train_words):
    if m1[0, iw] / m2[0, iw] < 2 and m2[0, iw] / m1[0, iw] < 2:
        print(iw, w, m1[0, iw], m2[0, iw])
print("Negative words")
for iw, w in enumerate(train_words):
    if m1[0, iw] / m2[0, iw] > 2:
        print(iw, w, m1[0, iw], m2[0, iw])
print("Positive words")
for iw, w in enumerate(train_words):
    if m2[0, iw] / m1[0, iw] > 2:
        print(iw, w, m1[0, iw], m2[0, iw])

In [ ]:
def classify(st, f_class1, f_class2, vocabulary, prior1, prior2):
    words = st.split(' ')
    prob1, prob2 = prior1, prior2
    for w in words:
        w2 = w
        w2 = w2.replace("!", "").replace("?", "").replace(".", "").replace(",", "")
        if w2 in train_words:
            prob1 *= f_class1[0, vocabulary.index(w2)]
            prob2 *= f_class2[0, vocabulary.index(w2)]
    probs = np.array([prob1, prob2])
    probs /= probs.sum()
    return probs

In [ ]:
p = classify("ok", m1, m2, train_words, p_y1, p_y2)
print(np.argmax(p), p[np.argmax(p)])

In [ ]:
p = classify("i am so sad this is bad news", m1, m2, train_words, p_y1, p_y2)
print(np.argmax(p), p[np.argmax(p)])

In [261]:
p = classify("this is fucking great. this is the most amazing news i've heard in a while, i am so happy", m1, m2, train_words, p_y1, p_y2)
print(np.argmax(p), p[np.argmax(p)])

1 0.9803830890487799


In [24]:
p = classify("daskhj adsjkh ", m1, m2, train_words, p_y1, p_y2)
print(np.argmax(p), p[np.argmax(p)])

0 0.5003309143494389


In [10]:
len(neg), len(pos)

(301993, 302393)